In [1]:
!pip3 install torch
!pip3 install matplotlib

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
names_list = open('names.txt','r').read().split('\n')

In [3]:
print(names_list[:10])

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia', 'harper', 'evelyn']


In [4]:
char_to_int = {}
int_to_char = {}

char_to_int['.'] = 0
int_to_char[0] = '.'

for char_num in range(ord('a'),ord('z')+1):
    integer_representation = char_num-ord('a')+1
    char_to_int[chr(char_num)] = integer_representation
    int_to_char[integer_representation] = chr(char_num)


In [5]:
import random
random.seed(42)
random.shuffle(names_list)

In [333]:
#hyperparameters

embed_size = 4

state_size = 10

hidden_size = 25

vocab_size = 28

In [340]:
import math
char_vector_size = 2
input_contexts , input_labels = [[] for _ in range(5)], [[] for _ in range(5)]

for name in names_list:
    log_padded_length = math.ceil(math.log(len(name))/math.log(2))
    padded_length = int(2**log_padded_length)
    context = [char_to_int[char] for char in name]  + [0] + [vocab_size-1]*(padded_length-len(name)) 
    input_contexts[log_padded_length].append(context)
#   instead of using labels, we can compare the result to the character vectors to find a likely match

print('c',input_contexts[2])


c [[10, 15, 18, 9, 0], [16, 8, 9, 1, 0], [1, 11, 9, 0, 27], [5, 7, 1, 14, 0], [26, 1, 4, 1, 0], [2, 1, 18, 9, 0], [1, 18, 25, 15, 0], [9, 7, 15, 18, 0], [1, 11, 9, 14, 0], [1, 4, 1, 13, 0], [16, 15, 12, 15, 0], [1, 2, 2, 25, 0], [19, 21, 1, 14, 0], [6, 9, 14, 14, 0], [10, 5, 14, 1, 0], [4, 18, 21, 22, 0], [10, 1, 14, 9, 0], [11, 15, 12, 0, 27], [3, 1, 18, 1, 0], [4, 1, 10, 1, 0], [10, 5, 16, 0, 27], [10, 1, 24, 9, 0], [5, 12, 1, 4, 0], [14, 25, 12, 1, 0], [13, 1, 20, 20, 0], [16, 5, 14, 14, 0], [26, 5, 5, 0, 27], [5, 11, 1, 13, 0], [14, 9, 12, 9, 0], [3, 12, 15, 5, 0], [10, 15, 5, 25, 0], [13, 15, 13, 15, 0], [10, 1, 3, 11, 0], [13, 9, 19, 1, 0], [1, 22, 1, 14, 0], [13, 5, 9, 0, 27], [9, 12, 25, 0, 27], [12, 5, 24, 25, 0], [7, 5, 13, 1, 0], [23, 5, 19, 20, 0], [1, 4, 15, 13, 0], [9, 22, 22, 25, 0], [5, 19, 8, 1, 0], [8, 21, 5, 25, 0], [12, 15, 22, 5, 0], [1, 11, 21, 12, 0], [22, 5, 4, 8, 0], [20, 8, 5, 15, 0], [21, 18, 9, 5, 0], [22, 1, 19, 8, 0], [18, 5, 4, 0, 27], [4, 9, 5, 13, 0], [

In [337]:
word = input_contexts[2][1]

In [338]:
# parameters
all_char_embeds = torch.randn((vocab_size,embed_size))

all_char_embeds[vocab_size-1] = 10**10

w_eh = torch.randn((embed_size, hidden_size)) * 0.5

w_sh = torch.randn((state_size, hidden_size)) * 0.5
w_so = torch.randn((state_size, embed_size)) *0.5

w_hs = torch.randn((hidden_size, state_size)) *0.5

b_h = torch.zeros((hidden_size,))
b_s = torch.zeros((state_size,))
b_o = torch.zeros((embed_size,))

init_state = torch.randn((state_size,)) * 0.5

network_params = [init_state,w_eh,w_sh,w_so,w_hs,b_h,b_s,b_o,all_char_embeds]

for p in network_params:
    p.requires_grad = True

In [10]:
def process_state_and_embed(states , embeds, hiddens, outputs, idx):
    hiddens[idx] = torch.sigmoid((states[idx] @ w_sh) + (embeds[idx] @ w_eh) + b_h)
    states[idx+1] = hiddens[idx] @ w_hs + b_s
    outputs[idx] = states[idx] @ w_so + b_o

In [11]:
batch_size = 32

for i in range(len(input_contexts)):
    input_contexts[i] = torch.tensor(input_contexts[i])

In [12]:
print(input_contexts[2])


batch_indices = torch.randint(0,input_contexts[2].shape[0],(batch_size,))

batch = input_contexts[2][batch_indices]

batch

tensor([[10, 15, 18,  9,  0],
        [16,  8,  9,  1,  0],
        [ 1, 11,  9,  0,  0],
        ...,
        [26,  9, 15, 14,  0],
        [19,  9, 18,  1,  0],
        [15, 16,  1, 12,  0]])


tensor([[13,  1, 18, 25,  0],
        [ 5, 18,  5, 26,  0],
        [26,  9, 15, 14,  0],
        [14,  9,  3,  1,  0],
        [12,  5,  9,  0,  0],
        [13,  1, 25,  5,  0],
        [ 5,  4,  5, 18,  0],
        [14,  9,  3, 15,  0],
        [20, 15, 18,  9,  0],
        [ 9, 26,  5, 14,  0],
        [ 1,  9,  4,  5,  0],
        [15, 19, 12, 15,  0],
        [11,  4,  5, 14,  0],
        [ 1, 22,  1, 14,  0],
        [ 1,  9, 25,  1,  0],
        [11, 15, 22, 21,  0],
        [ 9, 19, 19,  1,  0],
        [20, 15, 14,  9,  0],
        [15, 18, 12,  1,  0],
        [18, 21,  1,  0,  0],
        [20, 15, 18,  1,  0],
        [10,  1,  1, 14,  0],
        [13,  9, 12,  1,  0],
        [11,  1,  5, 15,  0],
        [ 9, 18, 25,  0,  0],
        [12, 21, 16,  5,  0],
        [15, 19, 12, 15,  0],
        [18, 15, 14, 25,  0],
        [17, 21,  1, 14,  0],
        [11, 15,  1, 12,  0],
        [19,  8,  1,  4,  0],
        [ 7,  5, 14, 20,  0]])

In [329]:
def predict(word):
    word_length = len(word)

    embeds = all_char_embeds[word]

    states = [init_state]+[torch.randn((state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((embed_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,embed_size))

    for i in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,i)

    return states[-1] @ w_so + b_o


def get_next(text):
    word = [char_to_int[c] for c in text] +[0]

    pred = predict(word)

    pred = pred.view(1,embed_size)
    dists = torch.cdist(pred,all_char_embeds)
    dists = dists.view(-1)
    dists = torch.exp(-200*dists)
    dists/=dists.sum()
    sampled = (torch.multinomial(dists,1)).item()
    return int_to_char[sampled]

In [330]:
for _ in range(100):
    st = ''
    curr = 'a'
    while curr!='.':
        curr = get_next(st)
        st+=curr

    print(st)

jreo.
ksis.
kria.
jrll.
ka.
jaa.
jo.
mara.
jlainlnn.
ml.
nlaeen.
jleaioiinn.
krrsnl.
kiieiy.
jliiiinae.
kei.
krra.
ko.
krialeenlnnniaini.
kraeilai.
ke.
.
kleaii.
ml.
klns.
kira.
mnia.
josn.
jl.
k.
ki.
krrlsi.
jyeaiylilii.
myeannn.
j.
jrli.
jeeale.
kraaion.
mr.
klre.
kaialoe.
krr.
je.
krlaen.
krlale.
jola.
kt.
mraa.
kn.
krea.
mnial.
kl.
jrne.
jrae.
krrl.
merain.
joaaer.
krn.
mnra.
m.
kora.
jo.
jrreri.
krile.
koeilolnrilnl.
jr.
jeaalinn.
je.
koia.
jo.
mlaaiyar.
kols.
mlra.
mela.
kaien.
kee.
kll.
kaeanylnln.
jriaini.
mria.
meraio.
krreeoi.
jlii.
myr.
jyrrlrlntine.
mlnaitenlll.
jrea.
mi.
jdie.
joiai.
mreaio.
jlleeoernr.
koernnn.
kr.
mraalye.
krialyll.
jrea.
jnea.
korsi.
ky.


In [339]:
all_char_embeds

tensor([[ 3.8753e-03,  5.4769e-01,  1.0621e+00, -5.0705e-02],
        [-1.8335e-02, -1.8557e-01,  1.0205e+00, -8.9196e-01],
        [ 5.0151e-01, -8.4993e-01,  6.6879e-01, -4.0833e-01],
        [-2.9351e-01,  1.4378e+00, -1.0488e+00,  2.5779e-01],
        [ 7.5454e-01, -1.0164e+00,  7.0758e-01,  1.5730e-02],
        [-6.3033e-01, -1.7464e+00, -4.8980e-01,  4.4696e-01],
        [ 2.2910e-01, -8.9719e-01, -1.1045e+00,  3.8398e-01],
        [ 9.0431e-02, -8.9994e-04,  1.2682e+00, -6.6247e-01],
        [-1.2860e+00,  5.6658e-01,  3.7770e-01,  6.8527e-01],
        [-4.8086e-01, -1.2267e+00, -1.4658e+00,  1.4709e+00],
        [-4.4370e-01,  2.0908e-01, -5.0658e-01,  1.9692e-01],
        [-8.1071e-01, -1.2054e+00, -2.1000e-01,  5.3870e-01],
        [ 9.7762e-01, -1.0415e+00,  9.7162e-01,  1.1112e+00],
        [-3.5794e-01,  5.5248e-01,  7.0010e-01, -2.4301e-01],
        [-1.0604e+00, -9.8741e-01,  1.8702e+00, -1.1625e+00],
        [ 7.7801e-01,  1.6024e-01,  1.6526e-01, -1.3359e+00],
        

In [323]:
loss_tot = 0

#shit training loop
for i in range(10000):
    log_idx = random.randint(1,4)
    
    batch_indices = torch.randint(0,input_contexts[log_idx].shape[0],(batch_size,))

    batch = input_contexts[log_idx][batch_indices]

    word_length = len(batch[0])

    embeds = torch.transpose(all_char_embeds[batch],0,1)

    states = [init_state] + [torch.zeros((batch_size,state_size,)) for _ in range(word_length-1)]

    hiddens = [torch.zeros((batch_size,embed_size,)) for _ in range(word_length-1)]

    outputs = torch.zeros((word_length,batch_size,embed_size))

    for i in range(word_length-1):
        process_state_and_embed(states,embeds ,hiddens,outputs,i)

    outputs[-1] = states[-1] @ w_so + b_o
    
    distances = torch.cdist(outputs,all_char_embeds).sum(2,keepdim=True)

    loss = ((outputs-embeds)**2 / distances).sum()
    
    loss.backward()
    
    for p in network_params:
        p.data -= 1e-4*p.grad
        p.grad = None
        
    loss_tot+= loss/ (batch_size*word_length)
print(loss_tot/10000)

tensor(0.0025, grad_fn=<DivBackward0>)
